In [ ]:
import psutil
import time
import uuid
import redis
import argparse as ap
from datetime import datetime
import paho.mqtt.client as mqtt
import json

In [ ]:
import redis
import uuid

# Your credentials
device = 0
host = 'redis-16255.c267.us-east-1-4.ec2.cloud.redislabs.com'
port = 16255
username = 'default'
password = 'J44swf7CVJjEMIjy6tEYuFke5PxxG0kf'

# Connect to your Redis server
redis_client = redis.Redis(host=host, port=port, username=username, password=password)

# Check connection
is_connected = redis_client.ping()
print('Redis Connected:', is_connected)

# Set retention periods when creating time series


Redis Connected: True


In [ ]:
# Create an MQTT client
mqtt_client = mqtt.Client()
mqtt_topic = "s317786"

# MQTT broker details
mqtt_broker_host = 'mqtt.eclipseprojects.io'  # replace with your MQTT broker host
mqtt_broker_port = 1883  # replace with your MQTT broker port


# Callback function for when the MQTT client connects to the broker
def on_connect(client, userdata, flags, rc):
    print(f'Connected to MQTT broker with result code {rc}')
    client.subscribe(mqtt_topic)

# Callback function for when a message is received from the MQTT broker
def on_message(client, userdata, msg):
    try:
        # Decode the JSON message
        message_data = json.loads(msg.payload.decode())

        # Extract MAC address and battery data from the message
        mac_address = message_data.get('mac_address')
        events = message_data.get('events', [])

        try:
            redis_client.ts().create(f'{mac_address}:battery', retention_msecs=86400000)
        except redis.exceptions.ResponseError:
            pass
        try:
            redis_client.ts().create(f'{mac_address}:power', retention_msecs=86400000)
        except redis.exceptions.ResponseError:
            pass

        for event in events:
            timestamp = event.get('timestamp')
            battery_level = event.get('battery_level')
            power_plugged = event.get('power_plugged')

            # Store data in Redis TimeSeries
            redis_client.ts().add(f'{mac_address}:battery', timestamp, battery_level)
            redis_client.ts().add(f'{mac_address}:power', timestamp, power_plugged)

            print(f"Data stored for {mac_address} - Timestamp: {timestamp}, Battery Level: {battery_level}, Power Plugged: {power_plugged}")
    except json.JSONDecodeError as e:
        print(f"Failed to decode JSON message: {e}")

# Set the callbacks
mqtt_client.on_connect = on_connect
mqtt_client.on_message = on_message

# Connect to the MQTT broker
mqtt_client.connect(mqtt_broker_host, mqtt_broker_port, 1882)

# Start the MQTT loop in the background
mqtt_client.loop_start()

# Keep the notebook running
while True:
    time.sleep(1)

Connected to MQTT broker with result code 0
Data stored for 0x56a24219aeec - Timestamp: 1705491661193, Battery Level: 85, Power Plugged: 0
Data stored for 0x56a24219aeec - Timestamp: 1705491662199, Battery Level: 85, Power Plugged: 0
Data stored for 0x56a24219aeec - Timestamp: 1705491663204, Battery Level: 85, Power Plugged: 0
Data stored for 0x56a24219aeec - Timestamp: 1705491664210, Battery Level: 85, Power Plugged: 0
Data stored for 0x56a24219aeec - Timestamp: 1705491665215, Battery Level: 85, Power Plugged: 0
Data stored for 0x56a24219aeec - Timestamp: 1705491666217, Battery Level: 85, Power Plugged: 0
Data stored for 0x56a24219aeec - Timestamp: 1705491667221, Battery Level: 85, Power Plugged: 0
Data stored for 0x56a24219aeec - Timestamp: 1705491668223, Battery Level: 85, Power Plugged: 0
Data stored for 0x56a24219aeec - Timestamp: 1705491669224, Battery Level: 85, Power Plugged: 0
Data stored for 0x56a24219aeec - Timestamp: 1705491670227, Battery Level: 85, Power Plugged: 0
Data s

KeyboardInterrupt: 

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=1b0b0df8-8392-4edb-9f9f-6f8029fb1a2c' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>